In [3]:
### Reranking Hybrid Search Statergies
# Re-ranking is a second-stage filtering process in retrieval systems, especially in RAG pipelines, where we:
# 1. First use a fast retriever (like BM25, FAISS, hybrid) to fetch top-k documents quickly.
# 2. Then use a more accurate but slower model (like a cross-encoder or LLM) to re-score and reorder those documents by relevance to the query.
# 👉 It ensures that the most relevant documents appear at the top, improving the final answer from the LLM.

In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain_core.output_parsers import StrOutputParser 

c:\Users\hites\OneDrive\Desktop\AI\agentic-rag\hybrid-search-strategies\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
## load text file
loader=TextLoader("data/langchain_sample.txt")
raw_docs=loader.load()

# Split text into document chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(raw_docs)

In [7]:
## user query
query="How can i use langchain to build an application with memory and tools?"

In [9]:
## OpenAI Embedding
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

### FAISS and Huggingface model Embeddings
from langchain_community.vectorstores import FAISS

load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

embeddings=OpenAIEmbeddings()
vectorstore_openai=FAISS.from_documents(docs,embeddings)
retriever_openai=vectorstore_openai.as_retriever(search_kwargs={"k":8})

In [10]:
## prompt and use the llm
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=init_chat_model("groq:gemma2-9b-it")

In [11]:
# Prompt Template
prompt = PromptTemplate.from_template("""
You are a helpful assistant. Your task is to rank the following documents from most to least relevant to the user's question.

User Question: "{question}"

Documents:
{documents}

Instructions:
- Think about the relevance of each document to the user's question.
- Return a list of document indices in ranked order, starting from the most relevant.

Output format: comma-separated document indices (e.g., 2,1,3,0,...)
""")

In [13]:
retrieved_docs=retriever_openai.invoke(query)
retrieved_docs

[Document(id='f6b6a283-fcdc-4948-b135-a80d6b1ac7a2', metadata={'source': 'data/langchain_sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.'),
 Document(id='2fb6eb58-d445-490f-9bce-62ce55b6b2ed', metadata={'source': 'data/langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(id='d787611c-c932-48e1-bc6a-10b4768507f3', metadata={'source': 'data/langchain_sample.txt'}, page_content='LangChain integrates with many third-party services such as Op

In [14]:
chain=prompt| llm | StrOutputParser()
chain

PromptTemplate(input_variables=['documents', 'question'], input_types={}, partial_variables={}, template='\nYou are a helpful assistant. Your task is to rank the following documents from most to least relevant to the user\'s question.\n\nUser Question: "{question}"\n\nDocuments:\n{documents}\n\nInstructions:\n- Think about the relevance of each document to the user\'s question.\n- Return a list of document indices in ranked order, starting from the most relevant.\n\nOutput format: comma-separated document indices (e.g., 2,1,3,0,...)\n')
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001EBAD156540>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001EBAD1B9D30>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [15]:
doc_lines = [f"{i+1}. {doc.page_content}" for i, doc in enumerate(retrieved_docs)]
formatted_docs = "\n".join(doc_lines)

In [16]:
response=chain.invoke({"question":query,"documents":formatted_docs})
response

"2,1,5,6,3,0 \n \nHere's why:\n\n* **Document 2** directly addresses the user's question by outlining LangChain's core features, including memory and tools.\n* **Document 1**  focuses specifically on memory and tools, making it highly relevant.\n* **Document 5** explains FAISS, a tool often used in conjunction with LangChain for memory and retrieval.\n* **Document 6** delves into RAG, a technique that leverages tools and memory within LangChain.\n* **Document 3**  is less relevant as it focuses on model integrations, although it indirectly touches on the broader capabilities of LangChain.\n* **Document 0** is the least relevant as it discusses retrieval methods without explicitly connecting them to memory or tools in the context of LangChain. \n\n\n"

In [17]:
# Step 5: Parse and rerank
indices = [int(x.strip()) - 1 for x in response.split(",") if x.strip().isdigit()]
indices

[1, 0, 4, 5, 2]

In [18]:
reranked_docs = [retrieved_docs[i] for i in indices if 0 <= i < len(retrieved_docs)]
reranked_docs

[Document(id='2fb6eb58-d445-490f-9bce-62ce55b6b2ed', metadata={'source': 'data/langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(id='f6b6a283-fcdc-4948-b135-a80d6b1ac7a2', metadata={'source': 'data/langchain_sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\nMemory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.'),
 Document(id='baf50f66-c735-4410-b7c7-0bf8b366c3e4', metadata={'source': 'data/langchain_sample.txt'}, page_content='FAISS is a popular library used for fast approximate nearest n

In [19]:
# Step 6: Show results
print("\n📊 Final Reranked Results:")
for i, doc in enumerate(reranked_docs, 1):
    print(f"\nRank {i}:\n{doc.page_content}")


📊 Final Reranked Results:

Rank 1:
LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.

Rank 2:
LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.
Memory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.

Rank 3:
FAISS is a popular library used for fast approximate nearest neighbor search in high-dimensional spaces. It supports both flat and compressed indexes, which makes it scalable for large document stores.
Agents in LangChain are chains that use LLMs to decide which tools to use and in what order. This makes them suitable for multi-step tasks like question answering w